## Data population
<b> NASDAQ six month data </b><br/>
Here I use the manually downloaded data trends of top 10 high performing tech stocks in csv format. <br/>
All this data will be stored in MySQL database on GCP instance setup during the semester as part of the class<br/>

In [17]:
import time
import pandas as pd
from pandas_datareader import data as pdr
import pymysql.cursors
import yfinance as yf
yf.pdr_override()
conn = pymysql.connect(host='34.133.230.107', 
                       user='root', password='@Mizu1997@',db='stock_prices')
cursor = conn.cursor()


#### Data manipulation

In [2]:
amd_data = pd.read_csv('AMD6Month.csv')
amd_data['name']='Advanced Micro Devices'
amd_data['Symbol']='AMD'
amd_data['sector']='Technology'
amd_data['industry']='Semiconductors'
apple_data = pd.read_csv('Apple6Month.csv')
apple_data['name']='Apple Inc'
apple_data['Symbol']='AAPL'
apple_data['sector']='Technology'
apple_data['industry']='Computer Manufacturing'
meta_data = pd.read_csv('Meta6Month.csv')
meta_data['name']='Meta'
meta_data['Symbol']='META'
meta_data['sector']='Technology'
meta_data['industry']='Internet and Information Services'
amazon_data = pd.read_csv('Amazon6Month.csv')
amazon_data['name']='Amazon'
amazon_data['Symbol']='AMZN'
amazon_data['sector']='Consumer Discretionary'
amazon_data['industry']='Catalog/Specialty Distribution'
cisco_data = pd.read_csv('Cisco6Month.csv')
cisco_data['name'] = 'Cisco'
cisco_data['Symbol']='CSCO'
cisco_data['sector']='Telecommunications'
cisco_data['industry']='Computer Communications Equipment'
micr_data = pd.read_csv('Microsoft6Month.csv')
micr_data['name']= 'Microsoft'
micr_data['Symbol']='MSFT'
micr_data['sector']='Technology'
micr_data['industry']='Computer Software: Prepackaged Software'
netflix_data = pd.read_csv('Netflix6Month.csv')
netflix_data['name']='Netflix'
netflix_data['Symbol']='NFLX'
netflix_data['sector']='Consumer Discretionary'
netflix_data['industry']='Consumer Electronics/Video Chains'
qualcomm_data = pd.read_csv('Qualcomm6Month.csv')
qualcomm_data['name']='Qualcomm'
qualcomm_data['Symbol']='QCOM'
qualcomm_data['sector']='Technology'
qualcomm_data['industry']='Computer peripheral equipment'
starbucks_data = pd.read_csv('StarBucks6Month.csv')
starbucks_data['name']='Starbucks'
starbucks_data['Symbol']='SBUX'
starbucks_data['sector']='Consumer Discretionary'
starbucks_data['industry']='Beverages - Non-Alcoholic'
telsa_data = pd.read_csv('Tesla6Month.csv')
telsa_data['name']='Telsa'
telsa_data['Symbol']='TSLA'
telsa_data['sector']='Consumer Discretionary'
telsa_data['industry']='Auto Manufacturing'

#### Merging all ten companies dataframes into single dataframe

In [4]:
df = apple_data
df = df.append(amd_data, ignore_index = True)
df = df.append(meta_data, ignore_index = True)
df = df.append(amazon_data, ignore_index = True)
df = df.append(cisco_data, ignore_index = True)
df = df.append(micr_data, ignore_index = True)
df = df.append(netflix_data, ignore_index = True)
df = df.append(qualcomm_data, ignore_index = True)
df = df.append(starbucks_data, ignore_index = True)
df = df.append(telsa_data, ignore_index = True)
df.insert(0, 'exchange_id', 2)
df.shape

/var/folders/0c/w_d0cwvj37bfcy_nv6kyzvcw0000gp/T/ipykernel_41695/3744505209.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(amd_data, ignore_index = True)
/var/folders/0c/w_d0cwvj37bfcy_nv6kyzvcw0000gp/T/ipykernel_41695/3744505209.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(meta_data, ignore_index = True)
/var/folders/0c/w_d0cwvj37bfcy_nv6kyzvcw0000gp/T/ipykernel_41695/3744505209.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(amazon_data, ignore_index = True)
/var/folders/0c/w_d0cwvj37bfcy_nv6kyzvcw0000gp/T/ipykernel_41695/3744505209.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

(1270, 11)

In [35]:
df.head(3)

,exchange_id,Date,Close/Last,Volume,Open,High,Low,name,Symbol,sector,industry
0,2,11/29/2022,141.17,83763800,144.29,144.81,140.355,Apple Inc,AAPL,Technology,Computer Manufacturing
1,2,11/28/2022,144.22,69346520,145.14,146.64,143.38,Apple Inc,AAPL,Technology,Computer Manufacturing
2,2,11/25/2022,148.11,35195860,148.305,148.88,147.12,Apple Inc,AAPL,Technology,Computer Manufacturing


#### Inserting the data from dataframe into the MySQL database

In [36]:
cols=['exchange_id', 'Symbol', 'name', 'sector', 'industry']
df1 = df[cols]
for row in df1.itertuples(index=False):
    print('================')
    try:
        print(row)
        cursor.execute("INSERT INTO security (exchange_id, ticker, name, sector, industry) VALUES (%s, %s, %s, %s, %s)", row)
    except:
        # Assume that the exception is because sector
        # and/or industry are missing
        cursor.execute("INSERT INTO security (exchange_id, ticker, name) VALUES (%s, %s, %s)", row[:3])
conn.commit()

In [30]:
all_tickers = pd.read_sql("SELECT ticker, id FROM security where exchange_id=2", conn)
ticker_index = dict(all_tickers.to_dict('split')['data'])
tickers = list(ticker_index.keys())
print(all_tickers)
tickers

  ticker    id
0   TSLA  4429
1   AAPL  4430
2    AMD  4431
3   META  4432
4   AMZN  4433
5   NFLX  4434
6   CSCO  4435
7   MSFT  4436
8   QCOM  4437
9   SBUX  4438


/var/folders/0c/w_d0cwvj37bfcy_nv6kyzvcw0000gp/T/ipykernel_41695/2947804990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_tickers = pd.read_sql("SELECT ticker, id FROM security where exchange_id=2", conn)


['TSLA', 'AAPL', 'AMD', 'META', 'AMZN', 'NFLX', 'CSCO', 'MSFT', 'QCOM', 'SBUX']

In [52]:
#Convert date to different format
# 2022-11-27 18:34:42
df['Date']= pd.to_datetime(df.Date)
print(type(df['Date'].values[0]))
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

<class 'numpy.datetime64'>


In [53]:
df.head(2)

,exchange_id,Date,Close/Last,Volume,Open,High,Low,name,Symbol,sector,industry
0,2,2022-11-29 00:00:00,141.17,83763800,144.29,144.81,140.355,Apple Inc,AAPL,Technology,Computer Manufacturing
1,2,2022-11-28 00:00:00,144.22,69346520,145.14,146.64,143.38,Apple Inc,AAPL,Technology,Computer Manufacturing


In [58]:
req_cols = ['Date','Open','High','Low','Close/Last','Volume']
for ticker in tickers:
    filtered = df[df['Symbol']==ticker][req_cols].reset_index(drop=True)
    # Write to daily_price
    for row in filtered.itertuples():
        values = [2, ticker_index[ticker]] + list(row)
        values = values[:2]+values[3:]
        cursor.execute("INSERT INTO daily_price (data_vendor_id, ticker_id, price_date, open_price, high_price, low_price, close_price, volume) VALUES  (%s, %s, %s, %s, %s, %s, %s, %s)",tuple(values))
conn.commit()

## Experimenting with Alpha vantage API for live data

In [1]:
import requests
from datetime import datetime
import matplotlib.pyplot as plt

In [7]:
daily_adjusted = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=H144IFU0KMLV9OC1'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'IBM', '3. Last Refreshed': '2022-11-30', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2022-11-30': {'1. open': '146.19', '2. high': '149.64', '3. low': '145.67', '4. close': '148.9', '5. adjusted close': '148.9', '6. volume': '5964095', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-29': {'1. open': '145.91', '2. high': '147.1668', '3. low': '145.7', '4. close': '146.49', '5. adjusted close': '146.49', '6. volume': '2754744', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-28': {'1. open': '147.98', '2. high': '148.24', '3. low': '145.935', '4. close': '146.18', '5. adjusted close': '146.18', '6. volume': '3538696', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-25': {'1. open': '148.27', '2. high': '149.49', '3. low': '148.101', '4. close': '148.37', '5. adjusted close': 

In [8]:
data['Time Series (Daily)']

{'2022-11-30': {'1. open': '146.19',
  '2. high': '149.64',
  '3. low': '145.67',
  '4. close': '148.9',
  '5. adjusted close': '148.9',
  '6. volume': '5964095',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-29': {'1. open': '145.91',
  '2. high': '147.1668',
  '3. low': '145.7',
  '4. close': '146.49',
  '5. adjusted close': '146.49',
  '6. volume': '2754744',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-28': {'1. open': '147.98',
  '2. high': '148.24',
  '3. low': '145.935',
  '4. close': '146.18',
  '5. adjusted close': '146.18',
  '6. volume': '3538696',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-25': {'1. open': '148.27',
  '2. high': '149.49',
  '3. low': '148.101',
  '4. close': '148.37',
  '5. adjusted close': '148.37',
  '6. volume': '2075156',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-23': {'1. open': '149.1',
  '2. high': '150.46',
  '3. lo

In [11]:
intraday='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=H144IFU0KMLV9OC1'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'IBM', '3. Last Refreshed': '2022-11-30', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2022-11-30': {'1. open': '146.19', '2. high': '149.64', '3. low': '145.67', '4. close': '148.9', '5. adjusted close': '148.9', '6. volume': '5964095', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-29': {'1. open': '145.91', '2. high': '147.1668', '3. low': '145.7', '4. close': '146.49', '5. adjusted close': '146.49', '6. volume': '2754744', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-28': {'1. open': '147.98', '2. high': '148.24', '3. low': '145.935', '4. close': '146.18', '5. adjusted close': '146.18', '6. volume': '3538696', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-11-25': {'1. open': '148.27', '2. high': '149.49', '3. low': '148.101', '4. close': '148.37', '5. adjusted close': 

In [17]:
keys = data['Time Series (Daily)'].keys()
data['Time Series (Daily)']

{'2022-11-30': {'1. open': '146.19',
  '2. high': '149.64',
  '3. low': '145.67',
  '4. close': '148.9',
  '5. adjusted close': '148.9',
  '6. volume': '5964095',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-29': {'1. open': '145.91',
  '2. high': '147.1668',
  '3. low': '145.7',
  '4. close': '146.49',
  '5. adjusted close': '146.49',
  '6. volume': '2754744',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-28': {'1. open': '147.98',
  '2. high': '148.24',
  '3. low': '145.935',
  '4. close': '146.18',
  '5. adjusted close': '146.18',
  '6. volume': '3538696',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-25': {'1. open': '148.27',
  '2. high': '149.49',
  '3. low': '148.101',
  '4. close': '148.37',
  '5. adjusted close': '148.37',
  '6. volume': '2075156',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2022-11-23': {'1. open': '149.1',
  '2. high': '150.46',
  '3. lo